In [ ]:
from pyspark.sql import SparkSession
import os
import logging
import sys

# Set JAVA_HOME
JAVA_HOME = "/home/spcstunnel/.sdkman/candidates/java/current"
os.environ["JAVA_HOME"] = JAVA_HOME

# Proxy Configuration (if required)
os.environ['HTTP_PROXY'] = 'http://localhost:8080'
#os.environ['HTTPS_PROXY'] = 'http://localhost:8080'
os.environ['http_proxy'] = 'http://localhost:8080'
#os.environ['https_proxy'] = 'http://localhost:8080'
#os.environ['PYSPARK_PYTHON'] = sys.executable
#os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

os.environ['AWS_REGION'] = 'us-east-1'
#MINIO_USER = "admin"
#MINIO_PASSWORD = "password"

# Proxy Configuration (if required)
#os.environ['HTTP_PROXY'] = os.environ['http_proxy'] = 'http://localhost:8080/'

# Set AWS environment variables (if required)

os.environ['AWS_ACCESS_KEY_ID'] = "admin"
os.environ['AWS_SECRET_ACCESS_KEY'] = "password"



#spark.stop()



# Initialize SparkSession with updated dependencies
spark = SparkSession.builder \
    .appName('IcebergQueryExample') \
    .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2,org.apache.hadoop:hadoop-aws:3.4.1') \
    .config('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')  \
    .config('spark.driver.extraJavaOptions', 
        '-Dhttp.proxyHost=localhost -Dhttp.proxyPort=8080 -Dhttps.proxyHost=localhost -Dhttps.proxyPort=8080 -Daws.region=us-east-1') \
    .config('spark.executor.extraJavaOptions', 
        '-Dhttp.proxyHost=localhost -Dhttp.proxyPort=8080 -Dhttps.proxyHost=localhost -Dhttps.proxyPort=8080 -Daws.region=us-east-1') \
    .config('spark.sql.defaultCatalog', 'warehouse') \
    .config('spark.sql.catalog.warehouse', 'org.apache.iceberg.spark.SparkCatalog')  \
    .config('spark.sql.catalog.warehouse.type', 'rest') \
    .config('spark.sql.catalog.warehouse.use.ssl', 'false') \
    .config('spark.sql.catalog.warehouse.uri', 'http://10.0.0.11:8181') \
    .config("spark.sql.catalog.warehouse.http.proxy.host", 'localhost') \
    .config("spark.sql.catalog.warehouse.http.proxy.port", '8080') \
    .config('spark.sql.catalog.warehouse.warehouse', 's3a://warehouse/') \
    .config('spark.sql.catalog.warehouse.s3.endpoint', 'http://10.0.0.11:9000') \
    .config('spark.sql.catalog.warehouse.io-impl','org.apache.iceberg.aws.s3.S3FileIO') \
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
    .config('spark.sql.catalog.warehouse.client.auth.type', 'basic') \
    .config('spark.sql.catalog.warehouse.client.auth.user', 'admin') \
    .config('spark.sql.catalog.warehouse.client.auth.password', 'password') \
    .config('spark.sql.catalog.warehouse.s3.path-style-access', 'true') \
    .config('spark.hadoop.fs.s3a.endpoint', 'http://10.0.0.11:9000') \
    .config('spark.sql.catalog.warehouse.rest.sigv4-enabled','false') \
    .config('spark.hadoop.fs.s3a.access.key', 'admin') \
    .config('spark.hadoop.fs.s3a.secret.key', 'password') \
    .config('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false') \
    .config('spark.hadoop.fs.s3a.path.style.access', 'true') \
    .config('spark.hadoop.fs.s3a.proxy.host', 'localhost') \
    .config('spark.hadoop.fs.s3a.proxy.port', '8080') \
    .getOrCreate()


logging.basicConfig(
    level=logging.INFO,  # Set to DEBUG for more detailed logs
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger("IcebergSparkJob")
# Set the current catalog to 'warehouse'
#spark.sql("USE CATALOG warehouse")
logger.info(f"Spark Jars: {spark.sparkContext._jsc.sc().listJars()}")








In [ ]:

spark.sql("SHOW CATALOGS").show()
spark.sql("SHOW DATABASES").show()
spark.sql("SELECT * FROM nyc.taxis WHERE payment_type = 1 AND trip_distance > 40 ORDER BY tolls_amount DESC").show()

#spark.stop()